In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11618816382973941110, name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2848410366365328436
 physical_device_desc: "device: XLA_GPU device", name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11846432243758509959
 physical_device_desc: "device: XLA_CPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 8261982618
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16877946169269940383
 physical_device_desc: "device: 0, name: TITAN V, pci bus id: 0000:01:00.0, compute capability: 7.0"]

In [32]:
# Test with a simple computation
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib
from sklearn.preprocessing import Imputer, MinMaxScaler
from sklearn.model_selection import train_test_split

In [88]:
train_data = pd.read_csv('dataset-0510/train.csv')
test_data = pd.read_csv('./dataset-0510/test.csv')

total_price = train_data.total_price
train_data.drop(columns=['building_id', 'total_price'], inplace=True)
test_data.drop(columns=['building_id'], inplace=True)


col_train = list(train_data.columns)
col_train_bis = list(train_data.columns)
train_data

,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,parking_price,...,XIV_250,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN
0,8,21,18674,4,3,2,6271,2,NaN,NaN,...,21,58,1,157,1,2483,1,6011,1,34.469803
1,8,7,18800,5,1,2,7885,2,NaN,NaN,...,7,28,1,115,1,15872,1,32221,1,40.073573
2,8,7,19289,4,1,2,6028,2,NaN,NaN,...,27,78,1,212,1,15760,1,32228,1,54.462081
3,8,21,20385,24,0,2,18325,0,NaN,81138.889762,...,2,20,1,125,1,2568,1,7271,1,99.628966
4,1,21,20657,2,4,2,6880,2,NaN,NaN,...,2,18,1,47,1,2587,1,7442,1,124.131236
5,8,21,18394,5,1,2,6576,2,NaN,NaN,...,26,58,1,314,1,2725,1,7417,1,61.109908
6,8,7,20474,15,0,2,16559,0,NaN,61488.568950,...,1,65,1,252,1,11515,1,26867,1,196.690224
7,8,12,18185,8,3,2,11934,1,NaN,6721.823057,...,9,64,1,181,1,11456,1,19514,1,66.780308
8,8,21,18794,5,1,2,7640,2,NaN,NaN,...,14,116,1,203,1,2666,1,7014,1,40.349486
9,8,7,19927,10,3,2,9863,0,NaN,61488.568950,...,19,118,1,288,1,12945,1,32664,1,102.955013


## preprocess

In [80]:
imp = Imputer()
scaler = MinMaxScaler()

train_data = scaler.fit_transform(train_data)
train_data = imp.fit_transform(train_data)

test_data = scaler.fit_transform(test_data)
test_data = imp.fit_transform(test_data)

/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [81]:
train_data = pd.DataFrame(train_data,columns = col_train)
test_data = pd.DataFrame(test_data,columns = col_test)

In [84]:
COLUMNS = col_train
FEATURES = col_train_bis
LABEL = "total_price"

# Columns for tensorflow
feature_cols = [tf.contrib.layers.real_valued_column(k) for k in col_train]

# Training set and Prediction set with the features to predict
training_set = train_data[col_train]
prediction_set = total_price

# Train and Test 
x_train, x_test, y_train, y_test = train_test_split(train_data , prediction_set, test_size=0.33, random_state=42)
y_train = pd.DataFrame(y_train, columns = [LABEL])
training_set = pd.DataFrame(x_train, columns = FEATURES).merge(y_train, left_index = True, right_index = True)


# Training for submission
training_sub = training_set[col_train]

training_set.head()

,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,parking_price,...,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN,total_price
25799,0.7,0.500000,0.871132,0.107143,1.00,0.2,0.790212,1.0,0.007455,0.049246,...,0.001785,1.0,0.020502,1.0,0.147953,0.0,0.331225,0.0,0.266368,4.304371e+06
56172,0.7,0.555556,0.857423,0.142857,0.25,0.2,0.256038,1.0,0.007455,0.049246,...,0.107079,1.0,0.151025,1.0,0.243246,0.0,0.336608,0.0,0.003572,1.112528e+07
7098,0.7,0.222222,0.903643,0.464286,0.00,0.2,0.703823,0.0,0.007455,0.060259,...,0.030339,1.0,0.042449,1.0,0.685188,0.0,0.916631,0.0,0.081000,1.741839e+07
2602,0.7,0.555556,0.741089,0.214286,0.75,0.2,0.257407,1.0,0.007455,0.049246,...,0.044616,1.0,0.056310,1.0,0.270645,0.0,0.622361,0.0,0.085324,1.916897e+07
15335,0.7,0.222222,0.222875,0.035714,1.00,0.8,0.459812,1.0,0.007455,0.049246,...,0.364664,1.0,0.583598,1.0,0.356184,0.0,0.961433,0.0,0.000783,1.652993e+08


In [87]:
# Same thing but for the test set
y_test = pd.DataFrame(y_test, columns = [LABEL])
testing_set = pd.DataFrame(x_test, columns = FEATURES).merge(y_test, left_index = True, right_index = True)
testing_set.head()

,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,parking_price,...,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN,total_price
12628,0.7,0.222222,0.736780,0.142857,0.25,0.2,0.340814,1.0,0.007455,0.049246,...,0.012493,1.0,0.045625,1.0,0.282920,0.0,0.615792,0.0,0.033845,7.001245e+06
37730,0.7,0.222222,0.227967,0.142857,0.25,0.5,0.644373,1.0,0.007455,0.049246,...,0.005354,1.0,0.018481,1.0,0.335799,0.0,0.833871,0.0,0.142607,3.900504e+06
39991,0.7,1.000000,0.810811,0.535714,0.00,0.2,0.595287,0.0,0.007455,0.020997,...,0.015467,1.0,0.041871,1.0,0.211135,0.0,0.292901,0.0,0.100165,9.035287e+06
8525,0.7,0.555556,0.777517,0.214286,0.75,0.2,0.273883,1.0,0.007455,0.049246,...,0.033314,1.0,0.057465,1.0,0.237547,0.0,0.673368,0.0,0.122837,2.225729e+07
8279,0.7,1.000000,0.983157,0.392857,0.50,0.2,0.376503,1.0,0.007455,0.049246,...,0.062463,1.0,0.089807,1.0,0.172229,0.0,0.248555,0.0,0.002186,3.706121e+05
